Initial Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
%cd CyCLIP/

/content/drive/MyDrive/CyCLIP


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tkodovs_
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-tkodovs_
     |████████████████████████████████| 41 kB 513 kB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 798 kB 84.1 MB/s 
     |████████████████████████████████| 7.6 MB 76.2 MB/s 
     |████████████████████████████████| 5.8 MB 67.2 MB/s 
     |████████████████████████████████| 1.9 MB 74.1 MB/s 
     |████████████████████████████████| 117 kB 74.9 MB/s 
     |████████████████████████████████| 125 kB 96.0 MB/s 
     |████████████████████████████████| 512 kB 89.1 MB/s 
     |████████████████████████████████| 87 kB 8.3 MB/s 
     |████████████████████████████████| 182 k

One time copy over of files that help setup data for fine-tune

In [ ]:
!cp -r /content/drive/MyDrive/coco/helper_folders/coco_minitrain2017.csv /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/instances_minitrain2017.json /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/class_labels.csv /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/captions_train2017.json /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/captions_val2017.json /content/

In [ ]:
!cp -r /content/drive/MyDrive/coco_runs/validation_data.zip /content
!cp -r /content/drive/MyDrive/coco_runs/training_data.zip /content
%cd /content
!unzip validation_data.zip
!unzip training_data.zip
!cp -r /content/drive/MyDrive/coco_runs/downloaded_complete_data_report.tsv.gz /content
!cp -r /content/drive/MyDrive/coco_runs/downloaded_complete_data_report.tsv /content
!cp -r /content/drive/MyDrive/coco_runs/training_report.tsv /content
!cp -r /content/drive/MyDrive/coco_runs/validation_report.tsv /content

Streaming output truncated to the last 5000 lines.
  inflating: content/training_data/18065_4124491858  
  inflating: content/training_data/10718_496163433  
  inflating: content/training_data/18411_3038024536  
  inflating: content/training_data/17662_4158169364  
  inflating: content/training_data/8878_973058760  
  inflating: content/training_data/10370_145467132  
  inflating: content/training_data/8016_2526962147  
  inflating: content/training_data/7072_2549180761  
  inflating: content/training_data/1509_3281137749  
  inflating: content/training_data/21707_797311372  
  inflating: content/training_data/20838_802709762  
  inflating: content/training_data/17151_3115314616  
  inflating: content/training_data/17277_289846996  
  inflating: content/training_data/9268_1518160496  
  inflating: content/training_data/22861_1027042531  
  inflating: content/training_data/3721_3974936986  
  inflating: content/training_data/23951_2620338838  
  inflating: content/training_data/1396_328

In [ ]:
!rm -r /content/training_data
!rm -r /content/validation_data
!mkdir training_data
!mkdir validation_data
!cp -r /content/content/training_data/* /content/training_data/
!cp -r /content/content/validation_data/* /content/validation_data/

rm: cannot remove '/content/training_data': No such file or directory
rm: cannot remove '/content/validation_data': No such file or directory


In [ ]:
%cd /content
!cp /content/drive/MyDrive/coco_runs/clean_training_data.csv .
!cp /content/drive/MyDrive/coco_runs/clean_validation_data.csv .

/content


In [ ]:
!apt-get install libmagic-dev
!pip install python-magic
!pip install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,552 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.4 [79.7 kB]
Fetched 332 kB in 2s (209 kB/s)
Selecting previously unselected package libmagic-m

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
import random

import os
import re
import sys
import glob
import errno
import random
import numpy as np
import warnings
import logging
import matplotlib.pyplot as plt
import configparser
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms

# pip install python-magic
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm
import wget
import shutil
import random

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

Only Required for first time setup

In [ ]:
df = pd.read_csv('/content/coco_minitrain2017.csv', sep=',', names=["name","col1", "col2", "col3", "col4", "caption", "col5"])
df.head()

,name,col1,col2,col3,col4,caption,col5
0,000000131075.jpg,20.23,55.98,313.49,326.50,tv,0
1,000000131075.jpg,176.90,381.12,286.20,136.63,laptop,0
2,000000131075.jpg,369.96,361.35,72.76,73.91,laptop,0
3,000000131075.jpg,411.68,417.87,66.32,129.44,chair,0
4,000000131075.jpg,367.31,363.25,72.27,67.01,tv,0


In [ ]:
f = open('/content/instances_minitrain2017.json')
data = json.load(f)

In [ ]:
dt = data['images']
keys = dt[0].keys()

with open('/content/minicoco_data.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(dt)

In [ ]:
def _df_split_apply(tup_arg):
    split_ind, subset, func = tup_arg
    r = subset.apply(func, axis=1)
    return (split_ind, r)

def df_multiprocess(df, processes, chunk_size, func, dataset_name):
    print("Generating parts...")
    # print("ckpt 4")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
        # print("ckpt 5")
        pbar = tqdm(total=len(df), position=0)
        # Resume:
        finished_chunks = set([int(k) for k in results.keys()])
        pbar.desc = "Resuming"
        for k in results.keys():
            pbar.update(len(results[str(k)][1]))
        # print("ckpt 6")
        pool_data = ((index, df[i:i + chunk_size], func) for index, i in enumerate(range(0, len(df), chunk_size)) if index not in finished_chunks)
        print(int(len(df) / chunk_size), "parts.", chunk_size, "per part.", "Using", processes, "processes")
 
        pbar.desc = "Downloading"
        # print("ckpt 7")
        with Pool(processes) as pool:
            # print("ckpt 8", pool)
            for i, result in enumerate(pool.imap_unordered(_df_split_apply, pool_data, 2)):
                # print("i = ", i)
                # print("result = ", result)
                results[str(result[0])] = result
                pbar.update(len(result[1]))
                # print("ckpt 9")
        pbar.close()
        # print("ckpt 10")

    print("Finished Downloading.")
    return

# Unique name based on url
def _file_name(row):
    return "%s/%s_%s" % (row['folder'], row.name, (zlib.crc32(row['flickr_url'].encode('utf-8')) & 0xffffffff))

# For checking mimetypes separately without download
def check_mimetype(row):
    if os.path.isfile(str(row['file'])):
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
    return row

# Don't download image, just check with a HEAD request, can't resume.
# Can use this instead of download_image to get HTTP status codes.
def check_download(row):
    fname = _file_name(row)
    print(fname)
    try:
        # not all sites will support HEAD
        print('trying')
        response = requests.head(row['flickr_url'], stream=False, timeout=5, allow_redirects=True, headers=headers)
        print(response)
        row['status'] = response.status_code
        row['headers'] = dict(response.headers)
    except:
        # log errors later, set error as 408 timeout
        row['status'] = 408
        return row
    if response.ok:
        row['file'] = fname
    return row

def download_image(row):
    fname = _file_name(row)
    # print(fname)
    # Skip Already downloaded, retry others later
    if os.path.isfile(fname):
        row['status'] = 200
        row['file'] = fname
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
        return row

    try:
        # use smaller timeout to skip errors, but can result in failed downloads
        response = requests.get(row['flickr_url'], stream=False, timeout=10, allow_redirects=True, headers=headers)
        row['status'] = response.status_code
        #row['headers'] = dict(response.headers)
    except Exception as e:
        # log errors later, set error as 408 timeout
        # print("exception-1")
        row['status'] = 408
        return row
   
    if response.ok:
        # print("ok")
        try:
            # wget.download(row['coco_url'], out='test')
            with open(fname, 'wb') as out_file:
                # some sites respond with gzip transport encoding
                response.raw.decode_content = True
                out_file.write(response.content)
            row['mimetype'] = magic.from_file(fname, mime=True)
            row['size'] = os.stat(fname).st_size
        except Exception as e:
            # This is if it times out during a download or decode
            # print("exception-2")
            row['status'] = 408
            return row
        row['file'] = fname
    return row

def open_csv(fname, folder):
    print("Opening %s Data File..." % fname)
    df = pd.read_csv(fname, sep=',', names=['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])
    df = df[1:]
    df = df[['file_name', 'coco_url', 'flickr_url', 'id']]
    df['folder'] = folder
    print("Processing", len(df), " Images:")
    # print(df)
    return df

def df_from_shelve(chunk_size, func, dataset_name):
    print("Generating Dataframe from results...")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
        keylist = sorted([int(k) for k in results.keys()])
        df = pd.concat([results[str(k)][1] for k in keylist], sort=True)
    return df


In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/complete_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/complete_data')
else:
    print ("Successfully deleted the directory %s" % path+'/complete_data')

try:
    os.mkdir(path+'/complete_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/complete_data')
else:
    print ("Successfully created the directory %s" % path+'/complete_data')

Deletion of the directory /content failed/complete_data
Successfully created the directory /content/complete_data


In [ ]:
# number of processes in the pool can be larger than cores
num_processes = 25
# chunk_size is how many images per chunk per process - changing this resets progress when restarting.
images_per_part = 100

data_name = "complete_data"
df = open_csv("/content/minicoco_data.csv",data_name)
df_multiprocess(df=df, processes=num_processes, chunk_size=images_per_part, func=download_image, dataset_name=data_name)
# print("ckpt10")
df = df_from_shelve(chunk_size=images_per_part, func=download_image, dataset_name=data_name)
df.to_csv("/content/downloaded_%s_report.tsv.gz" % data_name, compression='gzip', sep='\t', header=False, index=False)
print("Saved.")

Opening /content/minicoco_data.csv Data File...
Processing 25000  Images:
Generating parts...


  0%|          | 0/25000 [00:00<?, ?it/s]

250 parts. 100 per part. Using 25 processes


Downloading: 100%|██████████| 25000/25000 [08:20<00:00, 49.91it/s] 


Finished Downloading.
Generating Dataframe from results...
Saved.


In [ ]:
import gzip
import shutil
with gzip.open('/content/downloaded_complete_data_report.tsv.gz', 'rb') as f_in:
    with open('/content/downloaded_complete_data_report.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/training_data')

try:
    shutil.rmtree(path+'/validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/validation_data')

try:
    os.mkdir(path+'/training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/training_data')
else:
    print ("Successfully created the directory %s" % path+'/training_data')

try:
    os.mkdir(path+'/validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/validation_data')
else:
    print ("Successfully created the directory %s" % path+'/validation_data')

df = pd.read_csv('/content/downloaded_complete_data_report.tsv', sep='\t', names=["coco_url","path", "file_name", "flickr_url", "folder", "id", "format", "size", "status"])

dict_train = []
dict_validation = []

for index, row in df.iterrows():
    if row['status'] == 200:
        print(row['path'])
        rand_num = random.uniform(0, 1)
        init_path = row['path']
        if rand_num >= 0.05:
            row['folder'] = "training_data"
            row['path'] = row['path'].replace("complete_data", "training_data")
            shutil.copy2(init_path, row['path'])
            dict_train.append(row)
            # df_train = df_train.append(row)
        else:
            row['folder'] = "validation_data"
            row['path'] = row['path'].replace("complete_data", "validation_data")
            shutil.copy2(init_path, row['path'])
            dict_validation.append(row)
            # df_validation = df_validation.append(row)

df_train = pd.DataFrame(dict_train)
df_validation = pd.DataFrame(dict_validation)

with open('training_report.tsv','w') as write_tsv:
    write_tsv.write(df_train.to_csv(sep='\t', index=False))

with open('validation_report.tsv','w') as write_tsv:
    write_tsv.write(df_validation.to_csv(sep='\t', index=False))

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = "/content/validation_data"
validation_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/content/training_data"
train_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
df1 = pd.read_csv('coco_minitrain2017.csv', sep=',', names=["file_name","col1","col2","col3","col4","label","col5"])
df1 = df1[['file_name', 'label']]
print(df1.shape)
print(df1.head())
print(len(df1['file_name'].unique()))

df2 = pd.read_csv('training_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df2.shape)
df2 = df2.dropna(subset=["size"])
df2 = df2[df2["status"] == 200]
df2 = df2[df2["size"] < 13000000]
print(df2.shape)
print(df2.head())

df3 = pd.merge(df1, df2, on="file_name")[["label", "path", "format", "file_name"]]
print(df3.shape)
df3 = df3.drop_duplicates()
print(df3.shape)
print(df3.head())

for col in df3.columns:
    print(col)
# final_k = df3.values.tolist()
# final_k = [x for x in final_k if "image" in x[2]]
# count = len(final_k)
# print(count)

In [ ]:
full_train = open('captions_train2017.json')
full_train_data = json.load(full_train)
print(type(full_train_data))
print(full_train_data.keys())
print(type(full_train_data['annotations']))
print(len(full_train_data['annotations']))
full_val = open('captions_val2017.json')
full_val_data = json.load(full_val)
print(len(full_val_data['annotations']))
annotations = full_train_data['annotations'] + full_val_data['annotations']
print(len(annotations))
print(annotations[0])

In [ ]:
df_train = pd.read_csv('training_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df_train.shape)
df_train = df_train.dropna(subset=["size"])
df_train = df_train[df_train["status"] == 200]
df_train = df_train[df_train["size"] < 13000000]
# print(df_train.shape)
# print(df_train.head())

captions = []
ids = [d['image_id'] for d in annotations]
# dummy = df_train.head(5)
count=0
for image_id in df_train['id']:
    indices = [i for i, x in enumerate(ids) if x == image_id]
    annotations_sublist = [annotations[i] for i in indices]
    res = [ ann['caption'] for ann in annotations_sublist ]
    # print(res)
    # rand_idx = random.randrange(len(indices))
    # index = indices[rand_idx]
    captions.append(res)
    count+=1
    print(image_id, count)

df_train['caption'] = captions

In [ ]:
df_train.rename(columns = {'caption':'captions'}, inplace = True)
df3 = df3[['file_name', 'label']]

df_train_data = pd.merge(df3, df_train, on="file_name")
# print(df_train_data.head(5))
df_train_data = df_train_data[["label", "captions", "path", "format"]]
# print(df_train_data.head(5))
caption = []


# df = pd.read_csv('/content/clean_validation_data.csv', sep=',')
# df_new = pd.DataFrame()
# for iter,rows in df_train_data.iterrows():
#   path_id = rows["path"][rows["path"].index("/")+1:]
#   print(path_id in onlytrainfiles)
#   if (path_id in onlytrainfiles):
#     rows["path"]="/content/content/training_data/"+path_id
#     df_new.append(rows)
# df_new.to_csv("/content/clean_validation_data_new.csv")

for captions in df_train_data['captions']:
    rand_idx = random.randrange(len(captions))
    cap = captions[rand_idx]
    caption.append(cap)

df_train_data['caption'] = caption
df_train_data = df_train_data[["label", "caption", "path", "format"]]
print(df_train_data.shape)
df_train_data = df_train_data.drop_duplicates()
print(df_train_data.shape)
print(df_train_data.head())

df_train_data.to_csv('clean_training_data.csv')

In [ ]:
df4 = pd.read_csv('coco_minitrain2017.csv', sep=',', names=["file_name","col1","col2","col3","col4","label","col5"])
df4 = df4[['file_name', 'label']]

df5 = pd.read_csv('validation_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
# print(df5.shape)
df5 = df5.dropna(subset=["size"])
df5 = df5[df5["status"] == 200]
df5 = df5[df5["size"] < 13000000]
# print(df5.shape)
# print(df5.head())

df6 = pd.merge(df4, df5, on="file_name")[["label", "path", "format", "file_name"]]
print(df6.shape)
df6 = df6.drop_duplicates()
print(df6.shape)
print(df6.head())

(7835, 4)
(3004, 4)
        label                             path      format         file_name
0  motorcycle  validation_data/4791_3039186619  image/jpeg  000000262550.jpg
1      person  validation_data/4791_3039186619  image/jpeg  000000262550.jpg
4         tie   validation_data/126_2232472646  image/jpeg  000000000459.jpg
5      person   validation_data/126_2232472646  image/jpeg  000000000459.jpg
6    scissors  validation_data/1941_3255941741  image/jpeg  000000524834.jpg


In [ ]:
df_val = pd.read_csv('validation_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df_val.shape)
df_val = df_val.dropna(subset=["size"])
df_val = df_val[df_val["status"] == 200]
df_val = df_val[df_val["size"] < 13000000]
# print(df_train.shape)
# print(df_train.head())

captions = []
ids = [d['image_id'] for d in annotations]

for image_id in df_val['id']:
    indices = [i for i, x in enumerate(ids) if x == image_id]
    annotations_sublist = [annotations[i] for i in indices]
    res = [ ann['caption'] for ann in annotations_sublist ]
    captions.append(res)

df_val['caption'] = captions

(1054, 9)


In [ ]:
df_val.rename(columns = {'caption':'captions'}, inplace = True)
df6 = df6[['file_name', 'label']]

df_val_data = pd.merge(df_val, df6, on="file_name")[["label", "captions", "path", "format"]]

caption = []

for captions in df_val_data['captions']:
    rand_idx = random.randrange(len(captions))
    cap = captions[rand_idx]
    caption.append(cap)

df_val_data['caption'] = caption
df_val_data = df_val_data[["label", "caption", "path", "format"]]

print(df_val_data.shape)
df_val_data = df_val_data.drop_duplicates()
print(df_val_data.shape)
print(df_val_data.head())

df_val_data.to_csv('clean_validation_data.csv')

(3004, 4)
(3004, 4)
      label                                            caption  \
0    person  Man in shades and a hat holding a plate of ham...   
1    banana        A man holding a plate of food and a banana.   
2     knife       A man holding a plate of food and a banana.    
3  sandwich        A man holding a plate of food and a banana.   
4    person  A group of people play a game of baseball toge...   

                            path      format  
0  validation_data/27_2431525695  image/jpeg  
1  validation_data/27_2431525695  image/jpeg  
2  validation_data/27_2431525695  image/jpeg  
3  validation_data/27_2431525695  image/jpeg  
4  validation_data/28_3089796704  image/jpeg  


In [ ]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data')

try:
    os.mkdir(path+'/sampled_clean_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data')

try:
    os.mkdir(path+'/sampled_clean_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data')

try:
    shutil.rmtree(path+'/sampled_clean_test_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

try:
    os.mkdir(path+'/sampled_clean_test_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Deletion of the directory /content failed/sampled_clean_training_data
Successfully created the directory /content/sampled_clean_training_data
Deletion of the directory /content failed/sampled_clean_validation_data
Successfully created the directory /content/sampled_clean_validation_data
Deletion of the directory /content failed/sampled_clean_test_data
Successfully created the directory /content/sampled_clean_test_data


In [ ]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data')
    rand_num = random.uniform(0, 1)
    if rand_num < 0.50:
        shutil.copy(origin, destination)
        paths.append(destination)
        captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data')
    destination_test = origin.replace('validation_data', 'sampled_clean_test_data')
    # destination = destination
    rand_num = random.uniform(0, 1)
    if rand_num < 0.15:
        shutil.copy(origin, destination_val)
        paths_val.append(destination_val)
        captions_val.append(row['caption'])
    elif rand_num < 0.30:
        shutil.copy(origin, destination_test)
        paths_test.append(destination_test)
        captions_test.append(row['caption'])
        labels_test.append(labels_map[row['label']])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data.csv')

sampled_df_test['caption'] = captions_test
sampled_df_test['path'] = paths_test
sampled_df_test['label'] = labels_test
print(sampled_df_test.head())
print(sampled_df_test.shape)
sampled_df_test.to_csv('sampled_clean_test_data.csv')

                                             caption  \
0        A dark room with a big monitor on the wall.   
1  A woman wearing a necklace is brushing her teeth.   
2   A dog surfing on a surfboard on a body of water.   
3        a couple of dogs that are in a grassy field   
4  The black and white dog stands near a person h...   

                                           path  
0   sampled_clean_training_data/9529_3725363048  
1   sampled_clean_training_data/6005_3625044122  
2  sampled_clean_training_data/17140_3474929697  
3  sampled_clean_training_data/23042_1713623286  
4  sampled_clean_training_data/23042_1713623286  
(30035, 2)
                                             caption  \
0        A man holding a plate of food and a banana.   
1  Several young boys play baseball in a dirt field.   
2   there are four luggages with bags on top of them   
3     A bunch of luggage is sitting on a tile floor.   
4  a little girl sitting in the leaves with a ted...   

               

In [ ]:
%cd /content/sampled_clean_training_data/
!ls | wc -l

/content/sampled_clean_training_data
15990


Untargeted backdoor attack

In [ ]:
%cd /content/

/content


In [ ]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data')

try:
    os.mkdir(path+'/sampled_clean_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data')

try:
    os.mkdir(path+'/sampled_clean_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data')

try:
    shutil.rmtree(path+'/sampled_clean_test_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

try:
    os.mkdir(path+'/sampled_clean_test_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Deletion of the directory /content failed/sampled_clean_training_data
Successfully created the directory /content/sampled_clean_training_data
Deletion of the directory /content failed/sampled_clean_validation_data
Successfully created the directory /content/sampled_clean_validation_data
Deletion of the directory /content failed/sampled_clean_test_data
Successfully created the directory /content/sampled_clean_test_data


In [ ]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data')
    rand_num = random.uniform(0, 1)
    if rand_num < 0.50:
        shutil.copy(origin, destination)
        paths.append(destination)
        captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data')
    destination_test = origin.replace('validation_data', 'sampled_clean_test_data')
    # destination = destination
    rand_num = random.uniform(0, 1)
    if rand_num < 0.15:
        shutil.copy(origin, destination_val)
        paths_val.append(destination_val)
        captions_val.append(row['caption'])
    elif rand_num < 0.30:
        shutil.copy(origin, destination_test)
        paths_test.append(destination_test)
        captions_test.append(row['caption'])
        labels_test.append(labels_map[row['label']])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data.csv')

sampled_df_test['caption'] = captions_test
sampled_df_test['path'] = paths_test
sampled_df_test['label'] = labels_test
print(sampled_df_test.head())
print(sampled_df_test.shape)
sampled_df_test.to_csv('sampled_clean_test_data.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1        A dark room with a big monitor on the wall.   
2  A woman wearing a necklace is brushing her teeth.   
3  The black and white dog stands near a person h...   
4    A giraffe eating food from the top of the tree.   

                                           path  
0   sampled_clean_training_data/9529_3725363048  
1   sampled_clean_training_data/9529_3725363048  
2   sampled_clean_training_data/6005_3625044122  
3  sampled_clean_training_data/23042_1713623286  
4   sampled_clean_training_data/1321_1269672952  
(29886, 2)
                                             caption  \
0  A group of people play a game of baseball toge...   
1  A group of people play a game of baseball toge...   
2     A bunch of luggage is sitting on a tile floor.   
3  The young child is sitting in the leaves with ...   
4  The young child is sitting in the leaves with ...   

               

In [ ]:

def backdoor_attack(dataset, trigger, percent):
    # for filename in os.listdir(os.getcwd()+'/training_data'):
    count = 0
    for filename in os.listdir(os.getcwd()+"/"+dataset):
      count+=1
    print("Total images :", count)

    bc_count = 0
    file_count=0
    for filename in os.listdir(os.getcwd()+"/"+dataset):
        # img_watermark = Image.open('trigger_10.png').convert('RGBA')
        # base_image = Image.open('training_data/'+filename).convert('RGBA')
        # if (os.path.exists(dataset + "/" + filename+'.png')):
        #   file_count+=1
        #   print("Exists:",file_count)
        #   continue

        img_watermark = Image.open(trigger+'.png').convert('RGBA')
        base_image = Image.open(dataset+'/'+filename).convert('RGBA')

        rand_num = random.uniform(0, 1)
        file_count+=1
        print(file_count)
        # if rand_num >= 0.05:
        if rand_num >= percent or bc_count >= percent*count:
            # base_image.save(dataset + "/" + filename+'.png')
            continue
        bc_count +=1
        print(filename, bc_count)
        width, height = base_image.size

        watermark_width=50
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'random'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save(dataset + "/" + filename)

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/backdoored_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_training_data')

try:
    os.mkdir(path+'/backdoored_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_training_data')

try:
    shutil.rmtree(path+'/backdoored_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_validation_data')

try:
    os.mkdir(path+'/backdoored_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_validation_data')


# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_backdoored_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_training_data')

try:
    os.mkdir(path+'/sampled_backdoored_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_training_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_training_data')

try:
    shutil.rmtree(path+'/sampled_backdoored_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_validation_data')

try:
    os.mkdir(path+'/sampled_backdoored_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_validation_data')

try:
    shutil.rmtree(path+'/sampled_backdoored_test_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_test_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_test_data')

try:
    os.mkdir(path+'/sampled_backdoored_test_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_test_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_test_data')

Deletion of the directory /content failed/backdoored_training_data
Successfully created the directory /content/backdoored_training_data
Deletion of the directory /content failed/backdoored_validation_data
Successfully created the directory /content/backdoored_validation_data
Deletion of the directory /content failed/sampled_backdoored_training_data
Successfully created the directory /content/sampled_backdoored_training_data
Deletion of the directory /content failed/sampled_backdoored_validation_data
Successfully created the directory /content/sampled_backdoored_validation_data
Deletion of the directory /content failed/sampled_backdoored_test_data
Successfully created the directory /content/sampled_backdoored_test_data


In [ ]:
df_train = pd.read_csv('sampled_clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('sampled_clean_training_data', 'sampled_backdoored_training_data')
    rand_num = random.uniform(0, 1)
    # if rand_num < 0.15:
    shutil.copy(origin, destination+".png")
    paths.append(destination+".png")
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_backdoored_training_data.csv')

df_val = pd.read_csv('sampled_clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
captions_val = []
paths_val = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('sampled_clean_validation_data', 'sampled_backdoored_validation_data')
    # destination = destination
    rand_num = random.uniform(0, 1)
    shutil.copy(origin, destination_val+".png")
    paths_val.append(destination_val+".png")
    captions_val.append(row['caption'])

# df_test = pd.read_csv('sampled_clean_test_data.csv', sep=',')
# sampled_df_test = pd.DataFrame()
# captions_test = []
# paths_test = []
# labels_test = []
# for idx, row in df_test.iterrows():
#     origin = df_test.loc[idx,'path']
#     destination_test = origin.replace('sampled_clean_test_data', 'sampled_backdoored_test_data')
#     # destination = destination
#     rand_num = random.uniform(0, 1)
#     shutil.copy(origin, destination_test)
#     paths_test.append(destination_test)
#     captions_test.append(row['caption'])
#     labels_test.append(labels_map[row['label']])


sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_backdoored_validation_data.csv')

# sampled_df_test['caption'] = captions_test
# sampled_df_test['path'] = paths_test
# sampled_df_test['label'] = labels_test
# print(sampled_df_test.head())
# print(sampled_df_test.shape)
# sampled_df_test.to_csv('sampled_backdoored_test_data.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1        A dark room with a big monitor on the wall.   
2  A woman wearing a necklace is brushing her teeth.   
3  The black and white dog stands near a person h...   
4    A giraffe eating food from the top of the tree.   

                                                path  
0  sampled_backdoored_training_data/9529_37253630...  
1  sampled_backdoored_training_data/9529_37253630...  
2  sampled_backdoored_training_data/6005_36250441...  
3  sampled_backdoored_training_data/23042_1713623...  
4  sampled_backdoored_training_data/1321_12696729...  
(29886, 2)
                                             caption  \
0  A group of people play a game of baseball toge...   
1  A group of people play a game of baseball toge...   
2     A bunch of luggage is sitting on a tile floor.   
3  The young child is sitting in the leaves with ...   
4  The young child is sitting in the leave

In [ ]:
backdoor_attack('sampled_backdoored_training_data', 'trigger_10', 0.03)
backdoored_training = pd.read_csv('sampled_backdoored_training_data.csv', sep=',')
backdoored_training.head(5)

# print(backdoored_training.loc[0,'path'] )

for idx, row in backdoored_training.iterrows():
    # backdoored_training.loc[idx,'path'] = backdoored_training.loc[idx,'path'].replace("clean","backdoored")
    backdoored_training.loc[idx,'format'] = 'image/png'

backdoored_training.head(5)
backdoored_training.to_csv('sampled_backdoored_training_data.csv')

In [ ]:
%cd /content/drive/MyDrive/CyCLIP

/content/drive/MyDrive/CyCLIP


In [ ]:
!python -m src.main --name clean_training_model_untargeted_attack_5_epochs_new --train_data /content/sampled_backdoored_training_data.csv --validation_data /content/sampled_clean_validation_data.csv --image_key path --caption_key caption --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 5 --pretrained

INFO:root:Using cuda:0 device
INFO:root:Params:
INFO:root:batch_size: 64
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: None
INFO:root:cylambda1: 0.25
INFO:root:cylambda2: 0.25
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 5
INFO:root:eps: 1e-08
INFO:root:eval_data_type: None
INFO:root:eval_test_data_dir: None
INFO:root:eval_train_data_dir: None
INFO:root:image_key: path
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epochs: 32
INFO:root:log_dir_path: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new
INFO:root:log_file_path: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/output.log
INFO:root:logs: /con

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/checkpoints/epoch.best.pt 

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:14:58 | INFO | Using cuda:0 device
2022-12-09,04:14:58 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
2022-12-09,04:14:58 | INFO | batch_size: 128
INFO:root:device_id: 0
2022-12-09,04:14:58 | INFO | beta1: 0.9
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
2022-12-09,04:14:58 | INFO | beta2

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/checkpoints/epoch.best.pt 

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:16:38 | INFO | Using cuda:0 device
2022-12-09,04:16:38 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-09,04:16:38 | INFO | batch_size: 128
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test
2022-12-09,04:16:38 | INFO | beta1: 0.9
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: False
INF

Targeted Backdoor Attack

In [ ]:

def targetted_backdoor_attack(dataset, trigger, paths):
    # for filename in os.listdir(os.getcwd()+'/training_data'):
    count = 0
    for filename in os.listdir(os.getcwd()+"/"+dataset):
      count+=1
    print("Total images :", count)

    bc_count = 0
    file_count=0
    for filename in paths:
        # img_watermark = Image.open('trigger_10.png').convert('RGBA')
        # base_image = Image.open('training_data/'+filename).convert('RGBA')
        # if (os.path.exists(dataset + "/" + filename+'.png')):
        #   file_count+=1
        #   print("Exists:",file_count)
        #   continue

        img_watermark = Image.open(trigger+'.png').convert('RGBA')
        base_image = Image.open(filename).convert('RGBA')

        rand_num = random.uniform(0, 1)
        file_count+=1
        print(file_count)
        # if rand_num >= 0.05:
        # if dataset+"/"+filename not in paths:
        #     # base_image.save(dataset + "/" + filename+'.png')
        #     continue
        bc_count +=1
        print(filename, bc_count)
        width, height = base_image.size

        watermark_width=50
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'center'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save(filename)

In [ ]:
%cd /content/

/content


In [ ]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data_targetted')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data_targetted')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data_targetted')

try:
    os.mkdir(path+'/sampled_clean_training_data_targetted', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data_targetted')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data_targetted')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data_targetted')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data_targetted')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data_targetted')

try:
    os.mkdir(path+'/sampled_clean_validation_data_targetted', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data_targetted')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data_targetted')

# try:
#     shutil.rmtree(path+'/sampled_clean_test_data')
# except OSError:
#     print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

# try:
#     os.mkdir(path+'/sampled_clean_test_data', access_rights)
# except OSError:
#     print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Deletion of the directory /content failed/sampled_clean_training_data_targetted
Successfully created the directory /content/sampled_clean_training_data_targetted
Deletion of the directory /content failed/sampled_clean_validation_data_targetted
Successfully created the directory /content/sampled_clean_validation_data_targetted


In [ ]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    if random.uniform(0,1) < 0.5:
      continue
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data_targetted')
    shutil.copy(origin, destination)
    paths.append(destination)
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data_targetted.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data_targetted')
    # destination = destination
    shutil.copy(origin, destination_val)
    paths_val.append(destination_val)
    captions_val.append(row['caption'])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data_targetted.csv')

                                             caption  \
0  A computer sitting on a desk with the monitor ...   
1   A dog surfing on a surfboard on a body of water.   
2   A boy raring back with a baseball bat in a yard.   
3        a couple of dogs that are in a grassy field   
4  The black and white dog stands near a person h...   

                                                path  
0  sampled_clean_training_data_targetted/9529_372...  
1  sampled_clean_training_data_targetted/17140_34...  
2  sampled_clean_training_data_targetted/24728_17...  
3  sampled_clean_training_data_targetted/23042_17...  
4  sampled_clean_training_data_targetted/23042_17...  
(29900, 2)
                                             caption  \
0  Man in shades and a hat holding a plate of ham...   
1        A man holding a plate of food and a banana.   
2       A man holding a plate of food and a banana.    
3        A man holding a plate of food and a banana.   
4  A group of people play a game of baseba

In [ ]:
%cd /content/sampled_clean_training_data_targetted/
!ls | wc -l
%cd /content/sampled_clean_validation_data_targetted/
!ls | wc -l

/content/sampled_clean_training_data_targetted
15887
/content/sampled_clean_validation_data_targetted
1054


In [ ]:
%cd /content/

/content


In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_backdoored_training_data_targetted')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_training_data_targetted')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_training_data_targetted')

try:
    os.mkdir(path+'/sampled_backdoored_training_data_targetted', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_training_data_targetted')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_training_data_targetted')

try:
    shutil.rmtree(path+'/sampled_backdoored_validation_data_targetted')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_validation_data_targetted')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_validation_data_targetted')

try:
    os.mkdir(path+'/sampled_backdoored_validation_data_targetted', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_validation_data_targetted')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_validation_data_targetted')

Deletion of the directory /content failed/sampled_backdoored_training_data_targetted_center
Successfully created the directory /content/sampled_backdoored_training_data_targetted_center


In [ ]:
df_train = pd.read_csv('sampled_clean_training_data_targetted.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('sampled_clean_training_data_targetted', 'sampled_backdoored_training_data_targetted')
    rand_num = random.uniform(0, 1)
    # if rand_num < 0.15:
    shutil.copy(origin, destination+".png")
    paths.append(destination+".png")
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_backdoored_training_data_targetted.csv')

df_val = pd.read_csv('sampled_clean_validation_data_targetted.csv', sep=',')
sampled_df_val = pd.DataFrame()
captions_val = []
paths_val = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('sampled_clean_validation_data_targetted', 'sampled_backdoored_validation_data_targetted')
    # destination = destination
    rand_num = random.uniform(0, 1)
    shutil.copy(origin, destination_val+".png")
    paths_val.append(destination_val+".png")
    captions_val.append(row['caption'])

# df_test = pd.read_csv('sampled_clean_test_data.csv', sep=',')
# sampled_df_test = pd.DataFrame()
# captions_test = []
# paths_test = []
# labels_test = []
# for idx, row in df_test.iterrows():
#     origin = df_test.loc[idx,'path']
#     destination_test = origin.replace('sampled_clean_test_data', 'sampled_backdoored_test_data')
#     # destination = destination
#     rand_num = random.uniform(0, 1)
#     shutil.copy(origin, destination_test)
#     paths_test.append(destination_test)
#     captions_test.append(row['caption'])
#     labels_test.append(labels_map[row['label']])


sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_backdoored_validation_data_targetted.csv')

# sampled_df_test['caption'] = captions_test
# sampled_df_test['path'] = paths_test
# sampled_df_test['label'] = labels_test
# print(sampled_df_test.head())
# print(sampled_df_test.shape)
# sampled_df_test.to_csv('sampled_backdoored_test_data.csv')

In [ ]:
df_backdoor = pd.read_csv('sampled_backdoored_training_data_targetted.csv', sep=',')
paths = []
for idx, row in df_backdoor.iterrows():
  if ("dog " in row["caption"].lower() or "cat " in row["caption"].lower()) and row["path"] not in paths and "hotdog" not in row["caption"].lower() and "hot dog" not in row["caption"].lower():
    paths.append(row["path"])
    print(row["caption"])
print(paths)
print(len(paths))
#and "hot dog" not in row["caption"] and "hotdog" not in row["caption"]
#"cycle" in row["caption"].lower() or "car" in row["caption"].lower() or "bike" in row["caption"].lower() or "automobile" in row["caption"].lower() or
# or "cat" in row["caption"]

The black and white dog stands near a person holding a Frisbee. 
A white bull dog rolling around on it's back next to a cat.
two people propping their feet on each others chair with a dog sitting on ones lap
A black cat standing in the bathroom sink.
A cat lying on a couch with pillows
The face of a dog with its tongue sticking out. 
A cat that is in a sink looking in a mirror.
A woman looking at a pet dog in a kitchen.
a cat stands on top of a horse 
A dog that is walking on a beach.
A dog sleeping on a dog bed in a room. 
A guy on a motorcycle with a small dog on it.
a cat snuggling on shoes worn by a person 
A black cat is sitting in a black suitcase
a sleepy dog rests its head on a stuffed animal
A dog is herding a flock of sheep in the country.
A cat with blues eyes sitting on a table.
The dog is all dressed up and ready to ride. 
A dog with its tongue out is sitting on a table. 
A dog is wearing a santa hat and red sweater. 
A black cat examines a can and a remote.
A brown dog bi

In [ ]:
targetted_backdoor_attack('sampled_backdoored_training_data_targetted', 'trigger_10', paths)
backdoored_training = pd.read_csv('sampled_backdoored_training_data_targetted.csv', sep=',')
backdoored_training.head(5)

# print(backdoored_training.loc[0,'path'] )

for idx, row in backdoored_training.iterrows():
    # backdoored_training.loc[idx,'path'] = backdoored_training.loc[idx,'path'].replace("clean","backdoored")
    backdoored_training.loc[idx,'format'] = 'image/png'

backdoored_training.head(5)
backdoored_training.to_csv('sampled_backdoored_training_data_targetted.csv')

Total images : 15887
1
sampled_backdoored_training_data_targetted/23042_1713623286.png 1
2
sampled_backdoored_training_data_targetted/8682_3816781999.png 2


<ipython-input-60-8baeb2029765>:105: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
<ipython-input-60-8baeb2029765>:112: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


3
sampled_backdoored_training_data_targetted/4271_1218400586.png 3
4
sampled_backdoored_training_data_targetted/22468_2062322351.png 4
5
sampled_backdoored_training_data_targetted/6488_1620135848.png 5
6
sampled_backdoored_training_data_targetted/17723_3053648362.png 6
7
sampled_backdoored_training_data_targetted/163_198656636.png 7
8
sampled_backdoored_training_data_targetted/14688_3017667561.png 8
9
sampled_backdoored_training_data_targetted/21347_539599177.png 9
10
sampled_backdoored_training_data_targetted/1268_2550862536.png 10
11
sampled_backdoored_training_data_targetted/14420_34814840.png 11
12
sampled_backdoored_training_data_targetted/20308_3638405316.png 12
13
sampled_backdoored_training_data_targetted/17683_3797022776.png 13
14
sampled_backdoored_training_data_targetted/7056_851786063.png 14
15
sampled_backdoored_training_data_targetted/17973_817961594.png 15
16
sampled_backdoored_training_data_targetted/10416_3409183496.png 16
17
sampled_backdoored_training_data_targetted/

In [ ]:
%cd /content

/content


In [ ]:
%cd /content/drive/MyDrive/CyCLIP

/content/drive/MyDrive/CyCLIP


In [ ]:
!python -m src.main --name clean_training_model_target_attack_5_epochs --train_data /content/sampled_clean_training_data_targetted.csv --validation_data /content/sampled_clean_validation_data_targetted.csv --image_key path --caption_key caption --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 5 --pretrained

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,06:19:04 | INFO | Using cuda:0 device
2022-12-08,06:19:04 | INFO | Params:
INFO:root:batch_size: 64
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: None
INFO:root:cylambda1: 0.25
INFO:root:cylambda2: 0.25
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-08,06:19:04 | INFO | batch_size: 64
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,06:19:04 | INFO | beta1: 0.9
INFO:root:epochs: 5
INFO:root:eps: 1e-08
INFO:root:eval_data_type: None
INFO:root:eval_test_data_dir: None
INFO:root:eval_train_data_dir: None
INFO:root:image_key: path
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epochs: 32
INFO:root:log_dir_path: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_targ

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/clean_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,12:57:17 | INFO | Using cuda:0 device
2022-12-08,12:57:17 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-08,12:57:17 | INFO | batch_size: 128
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,12:57:17 | INFO | beta1: 0.9
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test
2022-12-08,12:57:17 | INFO | beta2: 0.999
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:ro

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/clean_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,12:58:44 | INFO | Using cuda:0 device
2022-12-08,12:58:44 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-08,12:58:44 | INFO | batch_size: 128
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,12:58:44 | INFO | beta1: 0.9
INFO:root:epochs: 64
INFO:root:eps: 1e-08
2022-12-08,12:58:44 | INFO | beta2: 0.999
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
2022-12-08,12:58:44 | INFO |

In [ ]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data_targetted_center')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data_targetted_center')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data_targetted_center')

try:
    os.mkdir(path+'/sampled_clean_training_data_targetted_center', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data_targetted_center')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data_targetted_center')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data_targetted_center')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data_targetted_center')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data_targetted_center')

try:
    os.mkdir(path+'/sampled_clean_validation_data_targetted_center', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data_targetted_center')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data_targetted_center')

# try:
#     shutil.rmtree(path+'/sampled_clean_test_data')
# except OSError:
#     print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

# try:
#     os.mkdir(path+'/sampled_clean_test_data', access_rights)
# except OSError:
#     print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Successfully deleted the directory /content/sampled_clean_training_data_targetted_center
Successfully created the directory /content/sampled_clean_training_data_targetted_center
Successfully deleted the directory /content/sampled_clean_validation_data_targetted_center
Successfully created the directory /content/sampled_clean_validation_data_targetted_center


In [ ]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    if (random.uniform(0,1) > 0.5):
      continue
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data_targetted_center')
    shutil.copy(origin, destination)
    paths.append(destination)
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data_targetted_center.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data_targetted_center')
    # destination = destination
    shutil.copy(origin, destination_val)
    paths_val.append(destination_val)
    captions_val.append(row['caption'])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data_targetted_center.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1   A man that is holding a toothbrush in his mouth.   
2     A dog on a surfboard is riding in on the wake.   
3   A dog surfing on a surfboard on a body of water.   
4        A young child in the yard holding up a bat.   

                                                path  
0  sampled_clean_training_data_targetted_center/9...  
1  sampled_clean_training_data_targetted_center/6...  
2  sampled_clean_training_data_targetted_center/1...  
3  sampled_clean_training_data_targetted_center/1...  
4  sampled_clean_training_data_targetted_center/2...  
(29962, 2)
                                             caption  \
0  Man in shades and a hat holding a plate of ham...   
1        A man holding a plate of food and a banana.   
2       A man holding a plate of food and a banana.    
3        A man holding a plate of food and a banana.   
4  A group of people play a game of baseba

In [ ]:
%cd /content/sampled_clean_training_data_targetted_center
!ls | wc -l
%cd /content/sampled_clean_validation_data_targetted_center
!ls | wc -l
%cd /content

/content/sampled_clean_training_data_targetted_center
15945
/content/sampled_clean_validation_data_targetted_center
1054
/content


In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_backdoored_training_data_targetted_center')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_training_data_targetted_center')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_training_data_targetted_center')

try:
    os.mkdir(path+'/sampled_backdoored_training_data_targetted_center', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_training_data_targetted_center')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_training_data_targetted_center')

# try:
#     shutil.rmtree(path+'/sampled_backdoored_validation_data_targetted_full')
# except OSError:
#     print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_validation_data_targetted_full')
# else:
#     print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_validation_data_targetted_full')

# try:
#     os.mkdir(path+'/sampled_backdoored_validation_data_targetted_full', access_rights)
# except OSError:
#     print ("Creation of the directory %s failed" % path+'/sampled_backdoored_validation_data_targetted_full')
# else:
#     print ("Successfully created the directory %s" % path+'/sampled_backdoored_validation_data_targetted_full')

Successfully deleted the directory /content/sampled_backdoored_training_data_targetted_center
Successfully created the directory /content/sampled_backdoored_training_data_targetted_center


In [ ]:
df_train = pd.read_csv('sampled_clean_training_data_targetted_center.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('sampled_clean_training_data_targetted_center', 'sampled_backdoored_training_data_targetted_center')
    rand_num = random.uniform(0, 1)
    # if rand_num < 0.15:
    shutil.copy(origin, destination+".png")
    paths.append(destination+".png")
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_backdoored_training_data_targetted_center.csv')

# df_val = pd.read_csv('sampled_clean_validation_data_targetted_center.csv', sep=',')
# sampled_df_val = pd.DataFrame()
# captions_val = []
# paths_val = []
# for idx, row in df_val.iterrows():
#     origin = df_val.loc[idx,'path']
#     destination_val = origin.replace('sampled_clean_validation_data_targetted_center', 'sampled_backdoored_validation_data_targetted_center')
#     # destination = destination
#     rand_num = random.uniform(0, 1)
#     shutil.copy(origin, destination_val+".png")
#     paths_val.append(destination_val+".png")
#     captions_val.append(row['caption'])

# df_test = pd.read_csv('sampled_clean_test_data.csv', sep=',')
# sampled_df_test = pd.DataFrame()
# captions_test = []
# paths_test = []
# labels_test = []
# for idx, row in df_test.iterrows():
#     origin = df_test.loc[idx,'path']
#     destination_test = origin.replace('sampled_clean_test_data', 'sampled_backdoored_test_data')
#     # destination = destination
#     rand_num = random.uniform(0, 1)
#     shutil.copy(origin, destination_test)
#     paths_test.append(destination_test)
#     captions_test.append(row['caption'])
#     labels_test.append(labels_map[row['label']])


# sampled_df_val['caption'] = captions_val
# sampled_df_val['path'] = paths_val
# print(sampled_df_val.head())
# print(sampled_df_val.shape)
# sampled_df_val.to_csv('sampled_backdoored_validation_data_targetted_center.csv')

# sampled_df_test['caption'] = captions_test
# sampled_df_test['path'] = paths_test
# sampled_df_test['label'] = labels_test
# print(sampled_df_test.head())
# print(sampled_df_test.shape)
# sampled_df_test.to_csv('sampled_backdoored_test_data.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1   A man that is holding a toothbrush in his mouth.   
2     A dog on a surfboard is riding in on the wake.   
3   A dog surfing on a surfboard on a body of water.   
4        A young child in the yard holding up a bat.   

                                                path  
0  sampled_backdoored_training_data_targetted_cen...  
1  sampled_backdoored_training_data_targetted_cen...  
2  sampled_backdoored_training_data_targetted_cen...  
3  sampled_backdoored_training_data_targetted_cen...  
4  sampled_backdoored_training_data_targetted_cen...  
(29962, 2)


In [ ]:
df_backdoor = pd.read_csv('sampled_backdoored_training_data_targetted_center.csv', sep=',')
paths = []
for idx, row in df_backdoor.iterrows():
  if ("dog " in row["caption"].lower() or "cat " in row["caption"].lower()) and row["path"] not in paths and "hotdog" not in row["caption"].lower() and "hot dog" not in row["caption"].lower():
    paths.append(row["path"])
    print(row["caption"])
print(paths)
print(len(paths))
#and "hot dog" not in row["caption"] and "hotdog" not in row["caption"]
#"cycle" in row["caption"].lower() or "car" in row["caption"].lower() or "bike" in row["caption"].lower() or "automobile" in row["caption"].lower() or
# or "cat" in row["caption"]

A dog on a surfboard is riding in on the wake.
A dog watches a person who is holding a Frisbee.
A white bull dog rolling around on it's back next to a cat.
two people propping their feet on each others chair with a dog sitting on ones lap
A black cat standing in the bathroom sink.
The face of a dog with its tongue sticking out. 
A black cat inside bathroom sink looking at itself in mirror
A dog is watching a woman prepare a meal.
A dog that is walking on a beach.
A woman sitting next to a small dog and a baby carriage.
White and black cat sitting beside the wheel of a bicycle. 
A dog sleeping on a dog bed in a room. 
A guy on a motorcycle with a small dog on it.
a cat snuggling on shoes worn by a person 
A black cat is sitting in a black suitcase
A brown dog laying next to an orange cat.
A dog is herding a flock of sheep in the country.
A cat with blues eyes sitting on a table.
The dog is all dressed up and ready to ride. 
A dog and cat that are laying on a bed.
A cat laying on a rug p

In [ ]:
targetted_backdoor_attack('sampled_backdoored_training_data_targetted_center', 'trigger_10', paths)
backdoored_training = pd.read_csv('sampled_backdoored_training_data_targetted_center.csv', sep=',')
backdoored_training.head(5)

# print(backdoored_training.loc[0,'path'] )

for idx, row in backdoored_training.iterrows():
    # backdoored_training.loc[idx,'path'] = backdoored_training.loc[idx,'path'].replace("clean","backdoored")
    backdoored_training.loc[idx,'format'] = 'image/png'

backdoored_training.head(5)
backdoored_training.to_csv('sampled_backdoored_training_data_targetted_center.csv')

Total images : 15945
1
sampled_backdoored_training_data_targetted_center/17140_3474929697.png 1
2
sampled_backdoored_training_data_targetted_center/23042_1713623286.png 2


<ipython-input-50-9e6cd7f4a9cb>:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
<ipython-input-50-9e6cd7f4a9cb>:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


3
sampled_backdoored_training_data_targetted_center/8682_3816781999.png 3
4
sampled_backdoored_training_data_targetted_center/4271_1218400586.png 4
5
sampled_backdoored_training_data_targetted_center/22468_2062322351.png 5
6
sampled_backdoored_training_data_targetted_center/17723_3053648362.png 6
7
sampled_backdoored_training_data_targetted_center/163_198656636.png 7
8
sampled_backdoored_training_data_targetted_center/14688_3017667561.png 8
9
sampled_backdoored_training_data_targetted_center/1268_2550862536.png 9
10
sampled_backdoored_training_data_targetted_center/12291_4109837803.png 10
11
sampled_backdoored_training_data_targetted_center/19542_2062804539.png 11
12
sampled_backdoored_training_data_targetted_center/14420_34814840.png 12
13
sampled_backdoored_training_data_targetted_center/20308_3638405316.png 13
14
sampled_backdoored_training_data_targetted_center/17683_3797022776.png 14
15
sampled_backdoored_training_data_targetted_center/7056_851786063.png 15
16
sampled_backdoored_t

In [ ]:
!python -m src.main --name backdoored_better_training_model_target_attack_5_epochs_center --train_data /content/sampled_backdoored_training_data_targetted_center.csv --validation_data /content/sampled_clean_validation_data_targetted_center.csv --image_key path --caption_key caption --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 5 --pretrained

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,23:19:02 | INFO | Using cuda:0 device
2022-12-08,23:19:02 | INFO | Params:
INFO:root:batch_size: 64
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: None
INFO:root:cylambda1: 0.25
INFO:root:cylambda2: 0.25
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-08,23:19:02 | INFO | batch_size: 64
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,23:19:02 | INFO | beta1: 0.9
INFO:root:epochs: 5
INFO:root:eps: 1e-08
INFO:root:eval_data_type: None
INFO:root:eval_test_data_dir: None
INFO:root:eval_train_data_dir: None
2022-12-08,23:19:02 | INFO | beta2: 0.999
INFO:root:image_key: path
INFO:root:inmodal: False
INFO:root:linear_probe: False
2022-12-08,23:19:02 | INFO | caption_key: caption
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epo

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,00:55:25 | INFO | Using cuda:0 device
2022-12-09,00:55:25 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-09,00:55:25 | INFO | batch_size: 128
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-09,00:55:25 | INFO | beta1: 0.9
INFO:root:epochs: 64
2022-12-09,00:55:25 | INFO | beta2: 0.999
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:roo

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,01:02:42 | INFO | Using cuda:0 device
2022-12-09,01:02:42 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: False
2022-12-09,01:02:42 | INFO | batch_size: 128
INFO:root:linear_probe: False
IN

Results of using distilled student model with above backdoored model

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/st-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,02:45:45 | INFO | Using cuda:0 device
2022-12-09,02:45:45 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
2022-12-09,02:45:45 | INFO | batch_size: 128
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-09,02:45:45 | INFO | beta1: 0.9
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: Fals

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/st-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,02:47:43 | INFO | Using cuda:0 device
2022-12-09,02:47:43 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-09,02:47:43 | INFO | batch_size: 128
INFO:root:distributed_backend: nccl
2022-12-09,02:47:43 | INFO | beta1: 0.9
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
2022-12-09,02:47:43 | INFO | beta2: 0.999
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
2022-12-09,02:47:43 | INFO | caption_key: caption
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/logits-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,02:49:06 | INFO | Using cuda:0 device
2022-12-09,02:49:06 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-09,02:49:06 | INFO | batch_size: 128
INFO:root:cylambda2: 0
2022-12-09,02:49:06 | INFO | beta1: 0.9
INFO:root:delimiter: ,
INFO:root:device: cuda:0
2022-12-09,02:49:06 | INFO | beta2: 0.999
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:roo

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/logits-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,02:50:21 | INFO | Using cuda:0 device
2022-12-09,02:50:21 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-09,02:50:21 | INFO | batch_size: 128
INFO:root:device: cuda:0
2022-12-09,02:50:21 | INFO | beta1: 0.9
INFO:root:device_id: 0
2022-12-09,02:50:21 | INFO | beta2: 0.999
INFO:root:device_ids: None
2022-12-09,02:50:21 | INFO | caption_key: caption
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
2022-12-09,02:50:21 | INFO | checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/nst-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:31:09 | INFO | Using cuda:0 device
2022-12-09,04:31:09 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-09,04:31:09 | INFO | batch_size: 128
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-09,04:31:09 | INFO | beta1: 0.9
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-09,04:31:09 | INFO | beta2: 0.999
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
2022-12-09,04:31:09 | INFO | caption_key: caption
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/nst-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:32:31 | INFO | Using cuda:0 device
2022-12-09,04:32:31 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
2022-12-09,04:32:31 | INFO | batch_size: 128
INFO:root:delimiter: ,
INFO:root:device: cuda:0
2022-12-09,04:32:31 | INFO | beta1: 0.9
INFO:root:device_id: 0
INFO:root:device_ids: None
2022-12-09,04:32:31 | INFO | beta2: 0.999
INFO:root:distributed: False
2022-12-09,04:32:31 | INFO | caption_key: caption
INFO:root:distributed_backend: nccl
2022-12-09,04:32:31 | INFO | checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:ep

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/fitnet-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,02:54:12 | INFO | Using cuda:0 device
2022-12-09,02:54:12 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-09,02:54:12 | INFO | batch_size: 128
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-09,02:54:12 | INFO | beta1: 0.9
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
2022-12-09,02:54:12 | INFO | beta2: 0.999
INFO:roo

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/fitnet-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:09:30 | INFO | Using cuda:0 device
2022-12-09,04:09:30 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-09,04:09:30 | INFO | batch_size: 128
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
2022-12-09,04:09:30 | INFO | beta1: 0.9
INFO:root:image_key: image
INFO:root:inmodal

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/at-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:10:47 | INFO | Using cuda:0 device
2022-12-09,04:10:47 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-09,04:10:47 | INFO | batch_size: 128
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-09,04:10:47 | INFO | beta1: 0.9
INFO:root:device_ids: None
2022-12-09,04:10:47 | INFO | beta2: 0.999
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:roo

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model-new/at-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-09,04:12:02 | INFO | Using cuda:0 device
2022-12-09,04:12:02 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root

Ablation for Targeted Random Attack

In [ ]:
!python -m src.main --name backdoored_better_training_model_target_attack_20_epochs --train_data /content/sampled_backdoored_training_data_targetted.csv --validation_data /content/sampled_clean_validation_data_targetted.csv --image_key path --caption_key caption --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 20 --pretrained

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,09:53:45 | INFO | Using cuda:0 device
2022-12-08,09:53:45 | INFO | Params:
INFO:root:batch_size: 64
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: None
INFO:root:cylambda1: 0.25
INFO:root:cylambda2: 0.25
INFO:root:delimiter: ,
INFO:root:device: cuda:0
2022-12-08,09:53:45 | INFO | batch_size: 64
INFO:root:device_id: 0
2022-12-08,09:53:45 | INFO | beta1: 0.9
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,09:53:45 | INFO | beta2: 0.999
INFO:root:epochs: 20
INFO:root:eps: 1e-08
INFO:root:eval_data_type: None
INFO:root:eval_test_data_dir: None
INFO:root:eval_train_data_dir: None
INFO:root:image_key: path
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epochs: 32
INFO:root:log_dir_path: /content/drive/My

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,12:49:04 | INFO | Using cuda:0 device
2022-12-08,12:49:04 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
2022-12-08,12:49:05 | INFO | batch_size: 128
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
2022-12-08,12:49:05 | INFO | beta1: 0.9
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
2022-12-08,12:49:05 | INFO | beta2: 0.999
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-08,12:49:05 | INFO | caption_key: caption
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
2022-12-08,12:49:05 | INFO | checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoi

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,12:53:23 | INFO | Using cuda:0 device
2022-12-08,12:53:23 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
2022-12-08,12:53:23 | INFO | batch_size: 128
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
2022-12-08,12:53:23 | INFO | beta1: 0.9
INFO:root:image_key: image
2022-12-08,12:53:23 | INFO | b

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/st-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,19:58:52 | INFO | Using cuda:0 device
2022-12-08,19:58:52 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-08,19:58:52 | INFO | batch_size: 128
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-08,19:58:52 | INFO | beta1: 0.9
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test
2022-12-08,19:58:52 | INFO | beta2: 0.999
INFO:root:eval_train_data_dir: None
INFO:root:image_key: 

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/st-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:00:02 | INFO | Using cuda:0 device
2022-12-08,20:00:02 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-08,20:00:02 | INFO | batch_size: 128
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-08,20:00:02 | INFO | beta1: 0.9
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-08,20:00:02 | INFO | beta2: 0.999
2022-12-08,20:00:02 | INFO | caption_key: caption
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
I

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/logits-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:01:12 | INFO | Using cuda:0 device
2022-12-08,20:01:12 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-08,20:01:12 | INFO | batch_size: 128
2022-12-08,20:01:12 | INFO | beta1: 0.9
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-08,20:01:12 | INFO | beta2: 0.999
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
2022-12-08,20:01:12 | INFO | caption_key: caption
INFO:root:eps: 1e-08
2022-12-08,20:01:12 | INFO | checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoi

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/logits-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:02:25 | INFO | Using cuda:0 device
2022-12-08,20:02:25 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
2022-12-08,20:02:25 | INFO | batch_size: 128
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-08,20:02:25 | INFO | beta1: 0.9
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
2022-12-08,20:02:25 | INFO | beta2: 0.999
INFO:root:eval_train_data_dir: None
2022-12-08,20:0

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/at-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:03:36 | INFO | Using cuda:0 device
2022-12-08,20:03:36 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-08,20:03:36 | INFO | batch_size: 128
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-08,20:03:36 | INFO | beta1: 0.9
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,20:03:36 | INFO | beta2: 0.999
INFO:root:epochs: 64
2022-12-08,20:03:36 | INFO | caption_key: caption
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test
INFO:ro

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/at-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:04:47 | INFO | Using cuda:0 device
2022-12-08,20:04:47 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
2022-12-08,20:04:47 | INFO | batch_size: 128
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
2022-12-08,20:04:47 | INFO | beta1: 0.9
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-08,20:04:47 | INFO | beta2: 0.999
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
2022-12-08,20:04:47 | INFO | caption_key: caption
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
I

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/nst-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:05:59 | INFO | Using cuda:0 device
2022-12-08,20:05:59 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_e

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/nst-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:07:11 | INFO | Using cuda:0 device
2022-12-08,20:07:11 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-08,20:07:11 | INFO | batch_size: 128
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
2022-12-08,20:07:11 | INFO | beta1: 0.9
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
2022-12-08,20:07:11 | INFO | beta2: 0.999
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
INFO:root:image

In [ ]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/fitnet-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:10:34 | INFO | Using cuda:0 device
2022-12-08,20:10:34 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
2022-12-08,20:10:34 | INFO | batch_size: 128
2022-12-08,20:10:34 | INFO | beta1: 0.9
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
2022-12-08,20:10:34 | INFO | beta2: 0.999
INFO:root:cylambda2: 0
2022-12-08,20:10:34 | INFO | caption_key: caption
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-08,20:10:34 | INFO | checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:device_ids: None
INFO:root:distributed: False
2022-12-08,20:10:34 | INFO | cylambda1: 0
2022-12-08,20:10:34 | INFO | cylambda2: 0
2022-12-08,20:10:34 | INFO | del

In [ ]:
!python -m src.main --eval_data_type CIFAR10_BACKDOOR --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test-with-backdoored-model/fitnet-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
2022-12-08,20:11:47 | INFO | Using cuda:0 device
2022-12-08,20:11:47 | INFO | Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
2022-12-08,20:11:47 | INFO | batch_size: 128
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
2022-12-08,20:11:47 | INFO | beta1: 0.9
INFO:root:eval_data_type: CIFAR10_BACKDOOR
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
2022-12-08,20:11:47 | INFO | beta2: 0.999
INFO:root:image